In [ ]:
import torch
import torch.nn as nn

from torch.optim import Adam, SGD
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

import random
from PIL import Image
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt


import pylab as pl
from IPython import display

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dades

Emprarem la Versió de 2012 de la base de dades PASCAL VOC [enllaç](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html) un dels conjunts de dades més coneguts. Nosaltres només en farem una petita exploració.

En aquest cas les etiquetes són a les imatges, és a dir la _label_ d'una imatge és una altra imatge d'un sol canal on cada pixel pot tenir 3 tipus de valor:

- 0: valor del fons
- 1 a 24: és un valor que indica que el pixel pertany a una classe (mirar la web del conjunt de dades)
- 255: pixel no etiquetat.


**Feina**

Carregar el conjunt de dades i seleccionar totes les imatges que contenen un moix (categoria _cat_). Per cada imatge que conté un moix, heu de posar tots els pixels que no són d'un moix de la imatge _label_ a 0 i els que són d'un moix a 1.

També heu de canviar la mida de les imatges a la següent dimensionalitat: $224x224$.

_Recomanació_: Abans de posar-vos a filtrar imatges inspeccionau els valors que aquestes tenen, la funció `np.unique` us pot servir d'ajuda.

In [ ]:

# Creating the dataset
train_dataset = datasets.VOCSegmentation(
    './datasets/',
    year='2012',
    download=True,
    image_set='train',
    transform=XXXX, #TODO 
)

valid_dataset = datasets.VOCSegmentation(
    './datasets/',
    year='2012',
    download=True,
    image_set='val',
    transform=XXXX, #TODO
)


In [ ]:
batch_size = 16 # no canviar
 #TODO 
train_loader = DataLoader(XXXX, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=2)
valid_loader = DataLoader(XXXX, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=2)

**Feina a fer**

Un cop teniu els conjunts de dades creats heu de comprovar que les imatges que es corresponen amb les etiquetes tenen la informació correcta, feis una visualització.

## Definició de la xarxa

Podem observar com es pot emprar l'orientació a objectes de **Python** per crear una xarxa de manera ordenada, és interessant analitzar aquest codi amb detall ja que en podem aprendre molt:

In [ ]:
#Credits: https://github.com/mateuszbuda/brain-segmentation-pytorch

from collections import OrderedDict

class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        
        ## CODER
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")
        
        ## DECODER
        self.upconv4 = nn.ConvTranspose2d(features * 16, features * 8, kernel_size=2, stride=2)
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        
        self.upconv3 = nn.ConvTranspose2d(features * 8, features * 4, kernel_size=2, stride=2)
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        
        self.upconv2 = nn.ConvTranspose2d(features * 4, features * 2, kernel_size=2, stride=2)
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        
        self.upconv1 = nn.ConvTranspose2d(features * 2, features, kernel_size=2, stride=2)
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(in_channels=features, out_channels=out_channels, kernel_size=1)
        
        

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)

        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        
        return torch.sigmoid(self.conv(dec1))

    
    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

## Entrenament

Per fer tasques de segmentació, una de les funcions de pèrdua que podem emprar és el _Diceloss_ (intersecció vs unió)

In [ ]:
class DiceLoss(nn.Module):

    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 0.0

    def forward(self, y_pred, y_true):
        assert y_pred.size() == y_true.size()
        y_pred = y_pred[:, 0].contiguous().view(-1)
        y_true = y_true[:, 0].contiguous().view(-1)
        intersection = (y_pred * y_true).sum()
        dsc = (2. * intersection + self.smooth) / (
            y_pred.sum() + y_true.sum() + self.smooth
        )
        return 1. - dsc

El bucle d'entrenament és diferent al que estau acostumats a veure en l'assignatura, s'assembla molt més als propis tutorials de _Pytorch_.

A més s'aprofita per introduir la visualització de resultats de forma dinàmica usant la llibreria [tqdm](https://github.com/tqdm/tqdm) i la llibreria _matplotlib_

In [ ]:
num_epochs = 100

model = UNet().to(device)

optim = Adam(model.parameters(), lr=1e-4)
criterion = DiceLoss() 

t_loss = np.zeros((num_epochs))
v_loss = np.zeros((num_epochs))

pbar = tqdm(range(1, num_epochs+1)) # tdqm permet tenir text dinàmic

for epoch in pbar:
    
    train_loss = 0 
    val_loss = 0  
    
    model.train()                                                  
    for batch_num, (input_img, target) in enumerate(train_loader, 1):   
        input_img= input_img.to(device)
        target = target.to(device)
        
        output = model(input_img)
        loss = criterion(output, target)
        loss.backward()                                            
        optim.step()                                               
        optim.zero_grad()     
        
        train_loss += loss.item()    
                                                       
    model.eval()   
    with torch.no_grad():                                          
        for input_img, target in valid_loader: 
            input_img = input_img.to(device)
            target = target.to(device)
            
            output = model(input_img)                                   
            loss = criterion(output, target)   
            val_loss += loss.item()  
    
    # RESULTATS
    train_loss /= len(train_loader)
    t_loss[epoch-1] = train_loss
    
    val_loss /= len(valid_loader)   
    v_loss[epoch-1] = val_loss
    
    # VISUALITZACIO DINAMICA
    plt.figure(figsize=(10,5))
    pl.plot(t_loss[:epoch-1], label="train")
    pl.plot(v_loss[:epoch-1], label="validation")
    pl.legend()
    pl.xlim(0,num_epochs)
    
    display.clear_output(wait=True)
    display.display(pl.gcf())
    plt.close()

    pbar.set_description(f"Epoch:{epoch} Training Loss:{train_loss} Validation Loss:{val_loss}")

Guardam el model, d'aquesta manera no es necessari fer l'entrenament a classe:

In [1]:
torch.save(model.state_dict(), "unet_pascal.pt")

NameError: name 'torch' is not defined

## Avaluació

Carregam el model

In [ ]:
mmodel =  UNet().to(device)
mmodel.load_state_dict(torch.load("unet_pascal.pt"))
mmodel.eval();

**Feina a fer**

Visualitzar exemples de segmentació.